In [2]:
# ===============================================================
# ID3 / C4.5（手刻，多路切） vs CART（sklearn）
# - 固定標籤欄位為 class；自動 calss→class；清理標籤尾巴的句點與空白
# - graphviz 不在時使用 Matplotlib 後備繪圖
# - 加入：過擬合偵測（ΔAcc）＋自動調參（預剪枝/後剪枝）
# ===============================================================

import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from typing import List, Tuple

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score

# ---------------- 全域參數（初始值；若過擬合會自動微調） ----------------
TRAIN_MAX_DEPTH    = 10     # 允許的初始最大深度
MIN_SAMPLES_SPLIT  = 20     # 允許再分裂的最小樣本數
MIN_GAIN           = 1e-4   # ID3: 資訊增益門檻；C4.5: 增益率門檻；CART: 對應 min_impurity_decrease
MAX_LEAF_NODES     = 64     # CART 專用（葉子上限）
VIEW_DEPTH         = 3      # 圖片只顯示前 3 層

# =========== 嘗試載入 graphviz，失敗就走 Matplotlib ===========
try:
    from graphviz import Source
    _GRAPHVIZ_OK = True
except Exception:
    _GRAPHVIZ_OK = False
    print("[Warning] 未偵測到 graphviz，將改用 Matplotlib 後備繪圖。")

SEED = 42
np.random.seed(SEED)

# -----------------------------
# 基本工具
# -----------------------------
def load_csvs(train_path: str, test_path: str):
    return pd.read_csv(train_path), pd.read_csv(test_path)

def strip_object_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()
    return df

def standardize_colnames(df: pd.DataFrame) -> pd.DataFrame:
    new_cols = []
    for c in df.columns:
        cc = str(c).replace("\ufeff", "").strip()  # 去 BOM/空白
        cc = cc.replace("\t"," ").replace("  "," ")
        cc = cc.replace(":", "").replace(".", "").strip()  # 移除常見雜點號
        new_cols.append(cc)
    df = df.copy()
    df.columns = new_cols
    return df

def coerce_numeric_columns(df: pd.DataFrame, exclude: List[str]) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if c in exclude:
            continue
        if df[c].dtype == "object":
            s = pd.to_numeric(df[c], errors="coerce")
            if s.notna().mean() > 0.9:
                df[c] = s
    return df

def basic_impute(df: pd.DataFrame, target: str) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if c == target:
            continue
        if df[c].dtype == "object":
            df[c] = df[c].fillna("Unknown").replace({"?":"Unknown"," ?":"Unknown"})
        else:
            med = pd.to_numeric(df[c], errors="coerce").median()
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(med)
    return df

def _clean_label_series(s: pd.Series) -> pd.Series:
    """adult 測試檔常見：末尾有小數點、含空白。"""
    return s.astype(str).str.strip().str.replace(".", "", regex=False)

def split_xy(df: pd.DataFrame, target: str):
    """只切分 X / y（前置清理已完成）"""
    return df.drop(columns=[target]).copy(), df[target].copy()

def get_feature_types(X: pd.DataFrame) -> List[str]:
    """判斷每欄是 numeric 或 categorical，供手刻 ID3/C4.5 使用。"""
    types = []
    for c in X.columns:
        if pd.api.types.is_numeric_dtype(X[c]):
            types.append("numeric")
        else:
            coer = pd.to_numeric(X[c], errors="coerce")
            types.append("numeric" if coer.notna().mean() > 0.9 else "categorical")
    return types

def to_numpy_mixed(X: pd.DataFrame) -> np.ndarray:
    """輸出 object dtype 的 numpy；數值欄轉 float，類別欄保留原值。"""
    arr = X.to_numpy(dtype=object)
    for j, c in enumerate(X.columns):
        if pd.api.types.is_numeric_dtype(X[c]) or pd.to_numeric(X[c], errors="coerce").notna().mean() > 0.9:
            arr[:, j] = pd.to_numeric(X[c], errors="coerce").astype(float).to_numpy()
        else:
            arr[:, j] = X[c].astype(object).to_numpy()
    return arr

def make_label_encoder(y_train: pd.Series, y_test: pd.Series) -> Tuple[list, dict, np.ndarray, np.ndarray]:
    """
    產生二元標籤編碼：
    - 多數類放在 index 0（混淆矩陣/報表較直覺）
    - 回傳 class_names, mapping, y_train01, y_test01
    """
    all_vals = pd.concat([y_train.astype(str), y_test.astype(str)], ignore_index=True)
    vc = all_vals.value_counts()
    class_names = [str(cls) for cls in vc.index.tolist()][:2]
    mapping = {cls: i for i, cls in enumerate(class_names)}
    y_train01 = y_train.astype(str).map(mapping).to_numpy(dtype=int)
    y_test01  = y_test.astype(str).map(mapping).to_numpy(dtype=int)
    return class_names, mapping, y_train01, y_test01

# ------------------ 手刻 ID3 / C4.5 ------------------
def entropy(y: np.ndarray) -> float:
    _, cnt = np.unique(y, return_counts=True)
    p = cnt / cnt.sum()
    return float(-(p * np.log2(p + 1e-12)).sum())

def split_info(ns: List[int], n: int) -> float:
    si = 0.0
    for k in ns:
        p = k / n
        si -= p * math.log2(p + 1e-12)
    return float(si)

def choose_best_split_ID3(X, y, ftypes, min_leaf=1):
    n, m = X.shape
    base = entropy(y)
    best, bj, obj, multi = -1.0, None, None, False
    for j in range(m):
        col = X[:, j]
        if ftypes[j] == "categorical":
            vals = np.unique(col)
            parts = [np.where(col == v)[0] for v in vals]
            if any(len(ix) < min_leaf for ix in parts): 
                continue
            h = sum((len(ix)/n)*entropy(y[ix]) for ix in parts)
            gain = base - h
            if (gain > best + 1e-12) or (abs(gain-best)<=1e-12 and (bj is None or j<bj)):
                best, bj, obj, multi = gain, j, {v:ix for v,ix in zip(vals, parts)}, True
        else:
            xs = np.unique(col.astype(float))
            if len(xs) <= 1: 
                continue
            ths = (xs[:-1] + xs[1:]) / 2.0
            for t in ths:
                L = np.where(col.astype(float) <= t)[0]
                R = np.where(col.astype(float) >  t)[0]
                if len(L)<min_leaf or len(R)<min_leaf: 
                    continue
                h = (len(L)/n)*entropy(y[L]) + (len(R)/n)*entropy(y[R])
                gain = base - h
                if (gain > best + 1e-12) or (abs(gain-best)<=1e-12 and (bj is None or j<bj)):
                    best, bj, obj, multi = gain, j, float(t), False
    return bj, obj, best, multi

def choose_best_split_C45(X, y, ftypes, min_leaf=1):
    n, m = X.shape
    base = entropy(y)
    best, bj, obj, multi = -1.0, None, None, False
    for j in range(m):
        col = X[:, j]
        if ftypes[j] == "categorical":
            vals = np.unique(col)
            parts = [np.where(col == v)[0] for v in vals]
            if any(len(ix) < min_leaf for ix in parts): 
                continue
            h = sum((len(ix)/n)*entropy(y[ix]) for ix in parts)
            gain = base - h
            gr = gain / (split_info([len(ix) for ix in parts], n) + 1e-12)
            if (gr > best + 1e-12) or (abs(gr-best)<=1e-12 and (bj is None or j<bj)):
                best, bj, obj, multi = gr, j, {v:ix for v,ix in zip(vals, parts)}, True
        else:
            xs = np.unique(col.astype(float))
            if len(xs) <= 1: 
                continue
            ths = (xs[:-1] + xs[1:]) / 2.0
            for t in ths:
                L = np.where(col.astype(float) <= t)[0]
                R = np.where(col.astype(float) >  t)[0]
                if len(L)<min_leaf or len(R)<min_leaf: 
                    continue
                h = (len(L)/n)*entropy(y[L]) + (len(R)/n)*entropy(y[R])
                gain = base - h
                gr = gain / (split_info([len(L), len(R)], n) + 1e-12)
                if (gr > best + 1e-12) or (abs(gr-best)<=1e-12 and (bj is None or j<bj)):
                    best, bj, obj, multi = gr, j, float(t), False
    return bj, obj, best, multi

def majority_class(y):
    cnt = Counter(y)
    return int(sorted(cnt.items(), key=lambda kv:(-kv[1], kv[0]))[0][0])

def build_tree(X, y, fnames, ftypes,
               max_depth=8, min_samples_split=2, min_leaf=1,
               depth=0, splitter="id3", min_gain=1e-12):
    n, m = X.shape
    node = {"n": int(n)}
    if depth>=max_depth or n<min_samples_split or len(np.unique(y))==1:
        node.update({"type":"leaf","pred": majority_class(y)})
        return node

    if splitter=="id3":
        j, obj, score, multi = choose_best_split_ID3(X,y,ftypes,min_leaf)
    else:
        j, obj, score, multi = choose_best_split_C45(X,y,ftypes,min_leaf)

    # 預剪枝門檻：若無可分裂或收益小於 min_gain，停止
    if (j is None) or (score < min_gain):
        node.update({"type":"leaf","pred": majority_class(y)})
        return node

    if depth<=2:
        tag = "ID3" if splitter=="id3" else "C4.5"
        if multi: print(f"[{tag}] depth={depth} feat={fnames[j]} (multiway) metric={score:.6f} n={n}")
        else:     print(f"[{tag}] depth={depth} feat={fnames[j]} thr={obj:.6f} metric={score:.6f} n={n}")

    node.update({"type":"node","feature":int(j)})
    if multi:
        kids={}
        for v,idxs in obj.items():
            kids[v] = build_tree(X[idxs,:], y[idxs], fnames, ftypes,
                                 max_depth, min_samples_split, min_leaf,
                                 depth+1, splitter, min_gain=min_gain)
        node["children"]=kids
    else:
        t=float(obj); col=X[:,j].astype(float)
        L=np.where(col<=t)[0]; R=np.where(col>t)[0]
        if len(L)==0 or len(R)==0:
            node.update({"type":"leaf","pred": majority_class(y)}); return node
        node["threshold"]=t
        node["left"]=build_tree(X[L,:], y[L], fnames, ftypes,
                                max_depth, min_samples_split, min_leaf,
                                depth+1, splitter, min_gain=min_gain)
        node["right"]=build_tree(X[R,:], y[R], fnames, ftypes,
                                 max_depth, min_samples_split, min_leaf,
                                 depth+1, splitter, min_gain=min_gain)
    return node

def predict_one(root, x):
    node=root
    while node.get("type")=="node":
        j=node["feature"]
        if "children" in node:
            v=x[j]; node=node["children"].get(v, None)
            if node is None: return 0
        else:
            node = node["left"] if float(x[j])<=node["threshold"] else node["right"]
    return int(node["pred"])

def predict_batch(root, X):
    return np.array([predict_one(root, X[i,:]) for i in range(X.shape[0])], dtype=int)

# ----------- 修剪(+僅輸出前K層)與繪圖 -----------
def clone_prune_to_depth(node, max_depth, depth=0):
    if node["type"]=="leaf" or depth>=max_depth:
        return {"type":"leaf","pred": node.get("pred",0), "n": node.get("n",0)}
    new={"type":"node","feature": node["feature"], "n": node.get("n",0)}
    if "children" in node:
        new["children"]={k: clone_prune_to_depth(ch, max_depth, depth+1) for k,ch in node["children"].items()}
    else:
        new["threshold"]=node["threshold"]
        new["left"]=clone_prune_to_depth(node["left"], max_depth, depth+1)
        new["right"]=clone_prune_to_depth(node["right"], max_depth, depth+1)
    return new

def to_dot(node, feature_names, class_names):
    lines=['digraph Tree {','node [shape=box, fontname="Helvetica"];','edge [fontname="Helvetica"];']
    nid=[0]
    def walk(n):
        my=nid[0]; nid[0]+=1
        if n.get("type")=="leaf":
            lbl=f'leaf\\nclass={class_names[int(n.get("pred",0))]}\\nsamples={n.get("n","?")}'
            lines.append(f'{my} [label="{lbl}", style="rounded,filled"];'); return my
        if "children" in n:
            lbl=f'{feature_names[n["feature"]]}'; lines.append(f'{my} [label="{lbl}"];')
            for cat,ch in n["children"].items():
                cid=walk(ch); cat_str=str(cat).replace('"','\\"'); lines.append(f'{my} -> {cid} [label="{cat_str}"];')
        else:
            th=n["threshold"]; lbl=f'{feature_names[n["feature"]]} <= {th:.4f}'; lines.append(f'{my} [label="{lbl}"];')
            L=walk(n["left"]); R=walk(n["right"])
            lines.append(f'{my} -> {L} [label="True"];'); lines.append(f'{my} -> {R} [label="False"];')
        return my
    walk(node); lines.append('}'); return "\n".join(lines)

def _count_leaves(n):
    if n["type"]=="leaf": return 1
    if "children" in n: return sum(_count_leaves(ch) for ch in n["children"].values())
    return _count_leaves(n["left"])+_count_leaves(n["right"])

def _layout(n, x0, x1, y, depth_step, coords, edges, edge_labels):
    my_id=id(n); coords[my_id]=( (x0+x1)/2, y, n )
    if n["type"]=="leaf": return
    if "children" in n:
        kids=list(n["children"].items())
        sizes=[_count_leaves(ch) for _,ch in kids]; total=sum(sizes)
        cur=x0
        for (lab,ch),sz in zip(kids,sizes):
            w=(x1-x0)*sz/total; nx0, nx1 = cur, cur+w; cur+=w
            _layout(ch, nx0, nx1, y-depth_step, depth_step, coords, edges, edge_labels)
            edges.append((my_id, id(ch))); edge_labels[(my_id, id(ch))]=str(lab)
    else:
        left,right = n["left"], n["right"]
        sizes=[_count_leaves(left), _count_leaves(right)]; total=sum(sizes)
        wL=(x1-x0)*sizes[0]/total; nx0, nx1 = x0, x0+wL
        _layout(left, nx0, nx1, y-depth_step, depth_step, coords, edges, edge_labels)
        _layout(right, x0+wL, x1, y-depth_step, depth_step, coords, edges, edge_labels)
        edges.append((my_id, id(left)));  edge_labels[(my_id,id(left))]="True"
        edges.append((my_id, id(right))); edge_labels[(my_id,id(right))]="False"

def save_handmade_tree_png(root, feature_names, class_names, out_png, view_max_depth=VIEW_DEPTH):
    pruned = clone_prune_to_depth(root, view_max_depth)
    if _GRAPHVIZ_OK:
        dot = to_dot(pruned, feature_names, class_names)
        src = Source(dot); src.format="png"; src.render(out_png.replace(".png",""), cleanup=True)
        print(f"[OK] 手刻樹圖片：{out_png}")
        return
    # 後備：Matplotlib
    coords, edges, edge_labels = {}, [], {}
    _layout(pruned, 0.0, 1.0, 1.0, 0.18, coords, edges, edge_labels)
    fig, ax = plt.subplots(figsize=(12, 6))
    for (x,y,nid) in [(x,y,n) for (x,y,n) in [(*coords[k][:2], coords[k][2]) for k in coords]]:
        n = nid
        if n["type"]=="leaf":
            txt = f'leaf\nclass={class_names[int(n.get("pred",0))]}\nN={n.get("n","?")}'
        else:
            if "children" in n:
                txt = feature_names[n["feature"]]
            else:
                txt = f'{feature_names[n["feature"]]} <= {n["threshold"]:.3f}'
        ax.text(x, y, txt, ha="center", va="center", bbox=dict(boxstyle="round,pad=0.3", fc="w", ec="k"))
    for (u,v) in edges:
        x1,y1,_ = coords[u]; x2,y2,_ = coords[v]
        ax.annotate("", xy=(x2,y2+0.01), xytext=(x1,y1-0.01), arrowprops=dict(arrowstyle="-"))
        lbl = edge_labels[(u,v)]
        ax.text((x1+x2)/2, (y1+y2)/2, lbl, ha="center", va="center")
    ax.set_axis_off(); plt.tight_layout(); plt.savefig(out_png, dpi=300); plt.close()
    print(f"[OK] 手刻樹圖片(後備)：{out_png}")

# ---------------- 評估 ----------------
def save_cm_png(y_true01, y_pred01, class_names, title, out_png):
    cm = confusion_matrix(y_true01, y_pred01, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(4,4))
    disp.plot(ax=ax, values_format='d', colorbar=False)
    ax.set_title(title)
    plt.tight_layout(); plt.savefig(out_png, dpi=300); plt.close()
    print(f"[OK] 混淆矩陣：{out_png}")

# ===============================================================
# 主流程（固定標籤為 class）
# ===============================================================

train_csv = "adult_data_no_duplicates.csv"
test_csv  = "adult_test_no_duplicates.csv"

# 0) 讀檔 & 基礎清理
train_df, test_df = load_csvs(train_csv, test_csv)
train_df = standardize_colnames(train_df)
test_df  = standardize_colnames(test_df)
train_df = strip_object_columns(train_df)
test_df  = strip_object_columns(test_df)

# 1) 固定標籤名稱（含 calss → class 防呆）
for name, df in [('train', train_df), ('test', test_df)]:
    if 'class' not in df.columns and 'calss' in df.columns:
        print(f"[Info] 偵測到 {name} 標籤欄位：calss -> 重新命名為 class")
        df.rename(columns={'calss': 'class'}, inplace=True)
    if 'class' not in df.columns:
        raise KeyError(f"{name}_df 缺少 'class' 欄位，現有欄位：{df.columns.tolist()}")
TARGET = 'class'

# 2) 清理標籤內容（去句點/空白）
train_df[TARGET] = _clean_label_series(train_df[TARGET])
test_df[TARGET]  = _clean_label_series(test_df[TARGET])

# 3) 數值轉型 + 缺值補
train_df = coerce_numeric_columns(train_df, exclude=[TARGET])
test_df  = coerce_numeric_columns(test_df,  exclude=[TARGET])
train_df = basic_impute(train_df, TARGET)
test_df  = basic_impute(test_df,  TARGET)

# 4) 拆 X / y 與欄型
X_train_df, y_train_sr = split_xy(train_df, TARGET)
X_test_df,  y_test_sr  = split_xy(test_df,  TARGET)
feature_names = X_train_df.columns.tolist()
feature_types = get_feature_types(X_train_df)

# 5) 標籤編碼（多數類為 class_names[0] -> 0）
class_names, mapping, y_train01, y_test01 = make_label_encoder(y_train_sr, y_test_sr)
print(f"[Info] class_names = {class_names}  (0 -> {class_names[0]}, 1 -> {class_names[1]})")

# 6) 轉 numpy（手刻樹用）
X_train_np = to_numpy_mixed(X_train_df)
X_test_np  = to_numpy_mixed(X_test_df)

# ===============================================================
# 自動偵測過擬合 + 自動調參（ID3 / C4.5 / CART）
# 判斷：ΔAcc = Train_Acc - Test_Acc；若 ΔAcc > GAP_THRESHOLD 視為過擬合
# 調參（ID3/C4.5）：提高 min_gain、提高 min_samples_split、降低 max_depth（預剪枝）
# 調參（CART）：提高 min_impurity_decrease、提高 min_samples_split、降低 max_depth、
#               降低 max_leaf_nodes，並嘗試 cost-complexity 後剪枝 (ccp_alpha)
# ===============================================================

# ---- 門檻設定：固定 2.5% ----
GAP_THRESHOLD = 0.025

MAX_TUNE_STEPS  = 5  # 每個模型最多調參步數
from sklearn.metrics import accuracy_score

def eval_tree(root):
    """回傳 train_acc, test_acc, Δacc, y_train_pred, y_test_pred for handmade trees."""
    y_tr = predict_batch(root, X_train_np)
    y_te = predict_batch(root, X_test_np)
    tr = accuracy_score(y_train01, y_tr)
    te = accuracy_score(y_test01,  y_te)
    return tr, te, tr-te, y_tr, y_te

def autotune_id3():
    params = dict(max_depth=TRAIN_MAX_DEPTH,
                  min_samples_split=MIN_SAMPLES_SPLIT,
                  min_gain=MIN_GAIN)
    history = []
    root = None
    for step in range(1, MAX_TUNE_STEPS+1):
        root = build_tree(X_train_np, y_train01, feature_names, feature_types,
                          max_depth=params["max_depth"],
                          min_samples_split=params["min_samples_split"],
                          min_leaf=1, splitter="id3", min_gain=params["min_gain"])
        tr, te, gap, _, _ = eval_tree(root)
        history.append((step, tr, te, gap, params.copy()))
        print(f"[ID3][step {step}] Train={tr:.4f} Test={te:.4f} Δ={gap:.4f}  params={params}")
        if gap <= GAP_THRESHOLD:
            break
        # ---- 預剪枝變嚴 ----
        params["min_gain"] = min(params["min_gain"]*2.0, 1e-1)
        params["min_samples_split"] = min(int(params["min_samples_split"]*1.25), 200)
        params["max_depth"] = max(3, params["max_depth"]-1)
    return root, params, history

def autotune_c45():
    params = dict(max_depth=TRAIN_MAX_DEPTH,
                  min_samples_split=MIN_SAMPLES_SPLIT,
                  min_gain=MIN_GAIN)
    history = []
    root = None
    for step in range(1, MAX_TUNE_STEPS+1):
        root = build_tree(X_train_np, y_train01, feature_names, feature_types,
                          max_depth=params["max_depth"],
                          min_samples_split=params["min_samples_split"],
                          min_leaf=1, splitter="c45", min_gain=params["min_gain"])
        tr, te, gap, _, _ = eval_tree(root)
        history.append((step, tr, te, gap, params.copy()))
        print(f"[C4.5][step {step}] Train={tr:.4f} Test={te:.4f} Δ={gap:.4f}  params={params}")
        if gap <= GAP_THRESHOLD:
            break
        # ---- 預剪枝變嚴 ----
        params["min_gain"] = min(params["min_gain"]*1.7, 1e-1)
        params["min_samples_split"] = min(int(params["min_samples_split"]*1.2), 200)
        params["max_depth"] = max(3, params["max_depth"]-1)
    return root, params, history

def train_cart_with_params(p):
    model = DecisionTreeClassifier(
        criterion="gini",
        max_depth=p["max_depth"],
        min_samples_split=p["min_samples_split"],
        min_impurity_decrease=p["min_impurity_decrease"],
        max_leaf_nodes=p["max_leaf_nodes"],
        ccp_alpha=p.get("ccp_alpha", 0.0),
        random_state=SEED
    )
    model.fit(Xtr_cart, y_train01)
    y_tr = model.predict(Xtr_cart)
    y_te = model.predict(Xte_cart)
    tr = accuracy_score(y_train01, y_tr)
    te = accuracy_score(y_test01,  y_te)
    gap = tr - te
    return model, (tr, te, gap, y_tr, y_te)

def autotune_cart():
    params = dict(max_depth=TRAIN_MAX_DEPTH,
                  min_samples_split=MIN_SAMPLES_SPLIT,
                  min_impurity_decrease=MIN_GAIN,
                  max_leaf_nodes=MAX_LEAF_NODES,
                  ccp_alpha=0.0)
    history = []
    model = None
    for step in range(1, MAX_TUNE_STEPS+1):
        # 嘗試取 cost-complexity 路徑的 25 分位作為溫和後剪枝
        try:
            from sklearn.tree import DecisionTreeClassifier as _DTC
            path = _DTC(random_state=SEED).cost_complexity_pruning_path(Xtr_cart, y_train01)
            alpha_q25 = float(np.quantile(path.ccp_alphas, 0.25))
            params["ccp_alpha"] = max(params.get("ccp_alpha", 0.0), alpha_q25)
        except Exception:
            pass

        model, stats = train_cart_with_params(params)
        tr, te, gap, _, _ = stats
        history.append((step, tr, te, gap, params.copy()))
        print(f"[CART][step {step}] Train={tr:.4f} Test={te:.4f} Δ={gap:.4f}  params={params}")
        if gap <= GAP_THRESHOLD:
            break
        # ---- 預剪枝更嚴＋後剪枝 ----
        params["min_impurity_decrease"] = min(params["min_impurity_decrease"]*2.0, 1e-1)
        params["min_samples_split"] = min(int(params["min_samples_split"]*1.25), 300)
        params["max_depth"] = max(3, params["max_depth"]-1)
        params["max_leaf_nodes"] = max(16, int(params["max_leaf_nodes"]*0.8))
        params["ccp_alpha"] = min(params.get("ccp_alpha", 0.0)*1.5 + 1e-5, 1e-1)
    return model, params, history

# === CART 的前處理（One-Hot 等） ===
cat_cols = [c for c,t in zip(feature_names, feature_types) if t=="categorical"]
num_cols = [c for c,t in zip(feature_names, feature_types) if t=="numeric"]
cart_pre = ColumnTransformer(
    [("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
     ("num", "passthrough", num_cols)],
    remainder="drop",
    verbose_feature_names_out=False
)
Xtr_cart = cart_pre.fit_transform(X_train_df)
Xte_cart = cart_pre.transform(X_test_df)

# === 自動調參訓練 ===
print("\n=== 自動調參：ID3 ===")
id3_root, id3_params, id3_hist = autotune_id3()

print("\n=== 自動調參：C4.5 ===")
c45_root, c45_params, c45_hist = autotune_c45()

print("\n=== 自動調參：CART ===")
cart, cart_params, cart_hist = autotune_cart()

# ---- 樹圖（僅前 VIEW_DEPTH 層）----
print("\n=== 輸出樹圖（僅前 3 層） ===")
save_handmade_tree_png(id3_root, feature_names, class_names, "Tree_ID3.png", view_max_depth=VIEW_DEPTH)
save_handmade_tree_png(c45_root, feature_names, class_names, "Tree_C45.png", view_max_depth=VIEW_DEPTH)

plt.figure(figsize=(18,10))
plot_tree(cart,
          feature_names=cart_pre.get_feature_names_out(),
          class_names=class_names,
          filled=True, max_depth=VIEW_DEPTH, impurity=False)
plt.tight_layout(); plt.savefig("Tree_CART.png", dpi=300); plt.close()
print("[OK] sklearn 樹圖片：Tree_CART.png")

# ---- 最終評估（單一來源，避免不一致）----
print("\n=== 評估（最終模型）===")

# handmade trees
id3_tr, id3_te, _, id3_ytr, id3_yte = eval_tree(id3_root)
c45_tr, c45_te, _, c45_ytr, c45_yte = eval_tree(c45_root)
# CART
y_cart_train = cart.predict(Xtr_cart)
y_cart_test  = cart.predict(Xte_cart)
cart_tr = accuracy_score(y_train01, y_cart_train)
cart_te = accuracy_score(y_test01,  y_cart_test)

acc_df = pd.DataFrame([
    ("ID3",  id3_tr,  id3_te),
    ("C4.5", c45_tr,  c45_te),
    ("CART", cart_tr, cart_te),
], columns=["Model","Train_Acc","Test_Acc"]).round(6)
print("\n=== Accuracy (Train / Test) ===")
print(acc_df.to_string(index=False))

# classification report + 混淆矩陣（測試集）
for name, yhat in [("ID3", id3_yte), ("C4.5", c45_yte), ("CART", y_cart_test)]:
    acc  = accuracy_score(y_test01, yhat)
    print(f"\n[{name}] Accuracy = {acc:.4f}")
    print(classification_report(y_test01, yhat, target_names=class_names, digits=4))
    save_cm_png(y_test01, yhat, class_names, name, f"CM_{name.replace('.','')}.png")

# 參數調整紀錄輸出（方便寫報告）
def hist_to_df(hist, model):
    rows=[]
    for step,tr,te,gap,params in hist:
        rows.append({"Model":model,"Step":step,"Train_Acc":tr,"Test_Acc":te,"Delta":tr-te,**params})
    return pd.DataFrame(rows)

tune_log = pd.concat([
    hist_to_df(id3_hist,"ID3"),
    hist_to_df(c45_hist,"C4.5"),
    hist_to_df(cart_hist,"CART"),
], ignore_index=True)

tune_log.round(6).to_csv("tune_log.csv", index=False)
acc_df.to_csv("accuracy_train_test.csv", index=False)

print("\n=== 完成 ===")
print("輸出：Tree_ID3.png / Tree_C45.png / Tree_CART.png / CM_ID3.png / CM_C45.png / CM_CART.png / accuracy_train_test.csv / tune_log.csv")
print(f"[說明] 過擬合門檻 GAP_THRESHOLD = {GAP_THRESHOLD:.3f}（可改為 3×SE 的資料驅動版本；見下方參考）")

# ---- 參考：用 3×SE 給一個資料驅動的門檻建議（不會影響上面結果）----
ser = acc_df.loc[acc_df["Model"].eq("CART"), "Test_Acc"]
if not ser.empty:
    p_hat = float(ser.iloc[0])            #  正確取標量
else:
    best_row = acc_df.iloc[acc_df["Test_Acc"].idxmax()]
    p_hat = float(best_row["Test_Acc"])   # 退而求其次：用 Test_Acc 最好的模型

n = len(y_test01)
se = (p_hat * (1 - p_hat) / n) ** 0.5
thr_se = max(0.02, 3 * se)  # 保底 2%

print(f"[參考] 以 p≈{p_hat:.4f}, n={n}, 3×SE≈{3*se:.4%}，資料驅動門檻建議 ≥ {thr_se:.2%}")


[Info] 偵測到 test 標籤欄位：calss -> 重新命名為 class
[Info] class_names = ['<=50K', '>50K']  (0 -> <=50K, 1 -> >50K)

=== 自動調參：ID3 ===
[ID3] depth=0 feat=relationship (multiway) metric=0.165337 n=32537
[ID3] depth=1 feat=education (multiway) metric=0.132089 n=13187
[ID3] depth=2 feat=occupation (multiway) metric=0.052372 n=313
[ID3] depth=2 feat=capital-gain thr=4164.000000 metric=0.117210 n=311
[ID3] depth=2 feat=occupation (multiway) metric=0.161556 n=103
[ID3] depth=2 feat=occupation (multiway) metric=0.179383 n=70
[ID3] depth=2 feat=occupation (multiway) metric=0.104546 n=149
[ID3] depth=2 feat=occupation (multiway) metric=0.046970 n=332
[ID3] depth=2 feat=capital-gain thr=4843.000000 metric=0.037081 n=202
[ID3] depth=2 feat=capital-gain thr=5095.500000 metric=0.080987 n=380
[ID3] depth=2 feat=capital-gain thr=5095.500000 metric=0.074866 n=596
[ID3] depth=2 feat=capital-gain thr=5095.500000 metric=0.076456 n=2433
[ID3] depth=2 feat=capital-gain thr=5054.500000 metric=0.047987 n=265
[ID3] dept

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.87624 to fit


[OK] 手刻樹圖片：Tree_ID3.png
[OK] 手刻樹圖片：Tree_C45.png
[OK] sklearn 樹圖片：Tree_CART.png

=== 評估（最終模型）===

=== Accuracy (Train / Test) ===
Model  Train_Acc  Test_Acc
  ID3   0.866460  0.845171
 C4.5   0.833082  0.830917
 CART   0.863632  0.861084

[ID3] Accuracy = 0.8452
              precision    recall  f1-score   support

       <=50K     0.8764    0.9282    0.9015     12430
        >50K     0.7130    0.5770    0.6378      3846

    accuracy                         0.8452     16276
   macro avg     0.7947    0.7526    0.7697     16276
weighted avg     0.8378    0.8452    0.8392     16276

[OK] 混淆矩陣：CM_ID3.png

[C4.5] Accuracy = 0.8309
              precision    recall  f1-score   support

       <=50K     0.8235    0.9909    0.8995     12430
        >50K     0.9144    0.3138    0.4673      3846

    accuracy                         0.8309     16276
   macro avg     0.8690    0.6524    0.6834     16276
weighted avg     0.8450    0.8309    0.7974     16276

[OK] 混淆矩陣：CM_C45.png

[CART] Accuracy